In [20]:
import pandas as pd 
import numpy as np

Start by loading in the raw data.

In [21]:
data = pd.read_csv('C:/Users/olsda/OneDrive/Documents/NU classes/Year 4/MMSS Thesis/PITGAN Implenetation/Data/usa_00001/census.csv')

Next we are going to drop the columns that are not relevant to the analysis.

In [22]:
relevant_cols = ['REGION', 'SEX', 'MARST', 'RACE', 'RACED', 'CITIZEN', 'EDUC','HCOVANY','HCOVPUB', 'EMPSTAT', 'POVERTY', 
                'INCTOT', 'FAMSIZE', 'AGE']

data = data[relevant_cols]

In [25]:
data.head()

,REGION,SEX,MARST,RACE,RACED,CITIZEN,EDUC,HCOVANY,HCOVPUB,EMPSTAT,POVERTY,INCTOT,FAMSIZE,AGE
0,South,1,1,2,200,0,5,2,1,1,99,3723,3,56
1,South,2,1,2,200,0,7,2,1,3,99,15107,3,61
2,South,2,6,2,200,0,1,2,2,0,99,9999999,3,8
3,South,1,4,2,200,0,7,1,1,1,284,37769,1,61
4,South,2,4,8,802,0,2,2,2,3,71,9496,1,52


In [5]:
data.shape

(15947624, 14)

Convert the regions column to the different regions. These regions are encoded as their overall geographical regions.

In [24]:
# Mapping dictionary
region_mapping = {
    11: 'Norhteast',
    12: 'Norhteast',
    13: 'Norhteast',
    21: 'Midwest',
    22: 'Midwest',
    23: 'Midwest',
    31: 'South',
    32: 'South',
    33: 'South',
    34: 'South',
    41: 'West',
    42: 'West',
    43: 'West',
    91: 'Unidentified',
    92: 'Unidentified',
    97: 'Unidentified',
    99: 'Unidentified'
}

# Applying the mapping
data['REGION'] = data['REGION'].map(region_mapping)

Remove records with a missing age

In [26]:
data = data[data['AGE']!=999]

Convert the sex, and drop all the rows where the sex is missing as this is a category used for identification.

In [27]:
# Mapping dictionary
sex_mapping = {
    1: 'Male',
    2: 'Female'
}

# Filter missing sex
data = data[data['SEX']!=9]

# Applying the mapping to the remaining values
data['SEX'] = data['SEX'].map(sex_mapping)

Convert the marital status column to reflect if someone is married, separated, or single. Also filter out column which are missing this values as it again is used for identification.

In [28]:
# Mapping dictionary
marital_mapping = {
    1: 'Married',
    2: 'Married',
    3: 'Separated',
    4: 'Separated',
    5: 'Widowed',
    6: 'Single'
}

# Filter missing marital status
data = data[data['MARST']!=9]

# Applying the mapping to the remaining values
data['MARST'] = data['MARST'].map(marital_mapping)

Convert the race colum to reflect some of the major race categories.

In [29]:
# Mapping dictionary
race_mapping = {
    1: 'White',
    2: 'Black',
    3: 'Native_Amer',
    4: 'Asian',
    5: 'Asian',
    6: 'Asian',
    7: 'Other',
    8: 'Multi',
    9: 'Multi'
}

# Applying the mapping to the remaining values
data['RACE'] = data['RACE'].map(race_mapping)

Convert the citizenship column to just reflect if someone is or is not a citizen. Drop missing values.

In [30]:
# Define the mapping from detailed status to 'Yes' or 'No'
citizenship_mapping = {
    0: "Yes",  
    1: "Yes",
    2: "Yes",
    3: "No",
    4: "No",  
    5: "No"
}

# Filter missing citizenship
data = data[(data['CITIZEN'] != 9) & (data['CITIZEN'] != 8)]

# Applying the mapping to the remaining values
data['CITIZEN'] = data['CITIZEN'].map(citizenship_mapping)

Convert the education to reflect the higheset level of educational achievement. 

In [31]:
# Function to categorize education
def categorize_education(code):
    code = int(code)  # Convert to integer for comparison
    if code < 6:
        return 'Not_Complete'
    elif 6 <= code <= 9:
        return 'High_School'
    elif code == 10:
        return 'Bachelors'
    elif code == 11:
        return 'Masters'

# Filter missing education
data = data[data['EDUC']!=99]

# Apply the function to the 'EDUC' column
data['EDUC'] = data['EDUC'].apply(categorize_education)

Convert the Health Care coverage to three categories, no insurance, public insurance and private insurance.

In [32]:
# Define the function to determine health coverage
def determine_health_coverage(row):
    if row['HCOVANY'] == 1:
        return 'No_Cov'
    elif row['HCOVANY'] == 2 and row['HCOVPUB'] == 1:
        return 'Priv_Cov'
    elif row['HCOVANY'] == 2 and row['HCOVPUB'] == 2:
        return 'Pub_Cov'

# Apply the function across rows to create the new column
data['HEALTHCOV'] = data.apply(determine_health_coverage, axis=1)

# Drop the two previous columns
data.drop(['HCOVANY', 'HCOVPUB'], axis=1, inplace=True)

Convert the employment status to clear categories. Filter out rows with missing values or those that are too young to be eligable.

In [33]:
# Define the mapping from detailed status
employment_mapping = {
    1: "Employed",  
    2: "Unemployed",
    3: "Not_in_LaborF"
}

# Filter missing employment status
data = data[(data['EMPSTAT'] != 9) & (data['EMPSTAT'] != 0)]

# Applying the mapping to the remaining values
data['EMPSTAT'] = data['EMPSTAT'].map(employment_mapping)

Convert the poverty column to those above or below the basic poverty line.

In [34]:
# Function to categorize education
def categorize_poverty(code):
    code = int(code)  # Convert to integer for comparison
    if code < 100:
        return 'Below'
    else:
        return 'Above'

# Apply the function to the 'EDUC' column
data['POVERTY'] = data['POVERTY'].apply(categorize_poverty)

Filter out income which is not reported (code = 0)

In [35]:
data = data[data['INCTOT']!=0]

Reorganize the data into the discrete columns first

In [39]:
relevant_cols = ['REGION', 'SEX', 'MARST', 'RACE', 'CITIZEN', 'EDUC','HEALTHCOV', 'EMPSTAT', 'POVERTY', 
                'INCTOT', 'FAMSIZE', 'AGE']

data = data[relevant_cols]

In [40]:
data.head()

,REGION,SEX,MARST,RACE,CITIZEN,EDUC,HEALTHCOV,EMPSTAT,POVERTY,INCTOT,FAMSIZE,AGE
0,South,Male,Married,Black,Yes,Not_Complete,Priv_Cov,Employed,Below,3723,3,56
1,South,Female,Married,Black,Yes,High_School,Priv_Cov,Not_in_LaborF,Below,15107,3,61
3,South,Male,Separated,Black,Yes,High_School,No_Cov,Employed,Above,37769,1,61
4,South,Female,Separated,Multi,Yes,Not_Complete,Pub_Cov,Not_in_LaborF,Below,9496,1,52
5,South,Male,Widowed,White,Yes,Bachelors,Pub_Cov,Not_in_LaborF,Above,14244,1,84


In [41]:
data.shape

(11591603, 12)

Finally store the data as a new csv for use later.

In [42]:
data.to_csv('C:/Users/olsda/OneDrive/Documents/NU classes/Year 4/MMSS Thesis/PITGAN Implenetation/Data/usa_00001/census_clean.csv', index=False)